# Clean movies metadata


## 1. Setup


In [33]:
import sys
sys.path.append('..')


In [34]:
import pandas as pd
import ast


from lib.types.dataset_type import DatasetType
from lib.types.source_type import SourceType

source: SourceType = SourceType.original


In [35]:
df = pd.read_csv(DatasetType.movies_metadata.path(source), low_memory=False)
df['id'] = df['id'].astype(str)
df.head(1)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


In [36]:
df.shape


(45465, 24)

In [37]:
df.columns.tolist()


['adult',
 'belongs_to_collection',
 'budget',
 'genres',
 'homepage',
 'id',
 'imdb_id',
 'original_language',
 'original_title',
 'overview',
 'popularity',
 'poster_path',
 'production_companies',
 'production_countries',
 'release_date',
 'revenue',
 'runtime',
 'spoken_languages',
 'status',
 'tagline',
 'title',
 'video',
 'vote_average',
 'vote_count']

## 2. Extract genres and belongs to collection


In [46]:

def apply_genre(row: pd.Series):
    genres = row['genres']

    parsed_json = ast.literal_eval(genres)

    genre_ids = "|".join(([str(i['id']) for i in parsed_json]))
    genre_names = "|".join(([i['name'] for i in parsed_json]))

    row['genre_ids'] = genre_ids
    row['genre_names'] = genre_names

    # fillna popularity with 0.0
    popularity = row['popularity']
    if(str(popularity) == 'nan'):
        row['popularity'] = 0.0

    btc: str = row['belongs_to_collection']
    if (btc != None):
        try:
            parsed_json_btc = ast.literal_eval(btc)
            btc_id = parsed_json_btc['id']
            btc_name = parsed_json_btc['name']
            row['btc_name'] = btc_name
            row['btc_id'] = str(btc_id)
        except:
            row['btc_name'] = None
            row['btc_id'] = None

    return row


cleaned_df = df.apply(lambda row: apply_genre(row), axis=1)
cleaned_df.drop(
    labels=[
        'adult',
        'belongs_to_collection',
        # 'budget',
        'genres',
        'homepage',
        # 'id',
        'imdb_id',
        'original_language',
        'original_title',
        'overview',
        # 'popularity',
        'poster_path',
        'production_companies',
        'production_countries',
        'release_date',
        # 'revenue',
        # 'runtime',
        'spoken_languages',
        'status',
        'tagline',
        # 'title',
        'video',
        # 'vote_average',
        # 'vote_count',
    ],
    axis=1,
    inplace=True
)


In [107]:
cleaned_df['genres'] = cleaned_df['genre_names'].str.split('|')

cleaned_df['genres1'] = cleaned_df['genres'].apply(lambda x: x[0])

cleaned_df['genres2'] = cleaned_df['genres'].apply(
    lambda x: x[1] if len(x) > 1 else x[0])

cleaned_df['genres3'] = cleaned_df['genres'].apply(
    lambda x: x[2] if len(x) > 2 else x[0])

cleaned_df['genres4'] = cleaned_df['genres'].apply(
    lambda x: x[3] if len(x) > 3 else x[0])

cleaned_df.drop('genres', axis=1, inplace=True)
cleaned_df.rename(columns={'genre_names': 'genres'}, inplace=True)
cleaned_df


,budget,id,popularity,revenue,runtime,title,vote_average,vote_count,genre_ids,genres,btc_name,btc_id,genres1,genres2,genres3,genres4
0,30000000,862,21.946943,373554033.0,81.0,Toy Story,7.7,5415.0,16|35|10751,Animation|Comedy|Family,Toy Story Collection,10194,Animation,Comedy,Family,Animation
1,65000000,8844,17.015539,262797249.0,104.0,Jumanji,6.9,2413.0,12|14|10751,Adventure|Fantasy|Family,None,None,Adventure,Fantasy,Family,Adventure
2,0,15602,11.712900,0.0,101.0,Grumpier Old Men,6.5,92.0,10749|35,Romance|Comedy,Grumpy Old Men Collection,119050,Romance,Comedy,Romance,Romance
3,16000000,31357,3.859495,81452156.0,127.0,Waiting to Exhale,6.1,34.0,35|18|10749,Comedy|Drama|Romance,None,None,Comedy,Drama,Romance,Comedy
4,0,11862,8.387519,76578911.0,106.0,Father of the Bride Part II,5.7,173.0,35,Comedy,Father of the Bride Collection,96871,Comedy,Comedy,Comedy,Comedy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45460,0,439050,0.072051,0.0,90.0,Subdue,4.0,1.0,18|10751,Drama|Family,None,None,Drama,Family,Drama,Drama
45461,0,111109,0.178241,0.0,360.0,Century of Birthing,9.0,3.0,18,Drama,None,None,Drama,Drama,Drama,Drama
45462,0,67758,0.903007,0.0,90.0,Betrayal,3.8,6.0,28|18|53,Action|Drama|Thriller,None,None,Action,Drama,Thriller,Action
45463,0,227506,0.003503,0.0,87.0,Satan Triumphant,0.0,0.0,,,None,None,,,,


## 3. Find average rating


In [108]:
rating_df = pd.read_csv(DatasetType.ratings.path(source))
rating_df.rename(columns={'userId': 'user_id', 'movieId': 'id'}, inplace=True)

avg_rating_df: pd.DataFrame = rating_df.groupby('id').agg(
    avg_rating=('rating', 'mean'),
    quantity=('user_id', 'count')
).reset_index()

avg_rating_df['id'] = avg_rating_df['id'].astype(str)
avg_rating_df['quantity'] = avg_rating_df['id'].astype(str)
avg_rating_df


,id,avg_rating,quantity
0,1,3.888157,1
1,2,3.236953,2
2,3,3.175550,3
3,4,2.875713,4
4,5,3.079565,5
...,...,...,...
45110,176267,4.000000,176267
45111,176269,3.500000,176269
45112,176271,5.000000,176271
45113,176273,1.000000,176273


## 4. Merge avg_rating_df with clean_df


In [109]:
avg_rating_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45115 entries, 0 to 45114
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          45115 non-null  object 
 1   avg_rating  45115 non-null  float64
 2   quantity    45115 non-null  object 
dtypes: float64(1), object(2)
memory usage: 1.0+ MB


In [110]:
cleaned_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 45465 entries, 0 to 45464
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   budget        45465 non-null  object 
 1   id            45465 non-null  object 
 2   popularity    45465 non-null  float64
 3   revenue       45461 non-null  float64
 4   runtime       45204 non-null  float64
 5   title         45461 non-null  object 
 6   vote_average  45461 non-null  float64
 7   vote_count    45461 non-null  float64
 8   genre_ids     45465 non-null  object 
 9   genres        45465 non-null  object 
 10  btc_name      4491 non-null   object 
 11  btc_id        4491 non-null   object 
 12  genres1       45465 non-null  object 
 13  genres2       45465 non-null  object 
 14  genres3       45465 non-null  object 
 15  genres4       45465 non-null  object 
dtypes: float64(5), object(11)
memory usage: 5.9+ MB


In [111]:
movie_df_cleaned = pd.merge(
    left=cleaned_df,
    right=avg_rating_df,
    on='id',
    how='left'
)
movie_df_cleaned


,budget,id,popularity,revenue,runtime,title,vote_average,vote_count,genre_ids,genres,btc_name,btc_id,genres1,genres2,genres3,genres4,avg_rating,quantity
0,30000000,862,21.946943,373554033.0,81.0,Toy Story,7.7,5415.0,16|35|10751,Animation|Comedy|Family,Toy Story Collection,10194,Animation,Comedy,Family,Animation,3.598930,862
1,65000000,8844,17.015539,262797249.0,104.0,Jumanji,6.9,2413.0,12|14|10751,Adventure|Fantasy|Family,None,None,Adventure,Fantasy,Family,Adventure,3.760163,8844
2,0,15602,11.712900,0.0,101.0,Grumpier Old Men,6.5,92.0,10749|35,Romance|Comedy,Grumpy Old Men Collection,119050,Romance,Comedy,Romance,Romance,NaN,NaN
3,16000000,31357,3.859495,81452156.0,127.0,Waiting to Exhale,6.1,34.0,35|18|10749,Comedy|Drama|Romance,None,None,Comedy,Drama,Romance,Comedy,NaN,NaN
4,0,11862,8.387519,76578911.0,106.0,Father of the Bride Part II,5.7,173.0,35,Comedy,Father of the Bride Collection,96871,Comedy,Comedy,Comedy,Comedy,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45460,0,439050,0.072051,0.0,90.0,Subdue,4.0,1.0,18|10751,Drama|Family,None,None,Drama,Family,Drama,Drama,NaN,NaN
45461,0,111109,0.178241,0.0,360.0,Century of Birthing,9.0,3.0,18,Drama,None,None,Drama,Drama,Drama,Drama,2.583333,111109
45462,0,67758,0.903007,0.0,90.0,Betrayal,3.8,6.0,28|18|53,Action|Drama|Thriller,None,None,Action,Drama,Thriller,Action,NaN,NaN
45463,0,227506,0.003503,0.0,87.0,Satan Triumphant,0.0,0.0,,,None,None,,,,,NaN,NaN


In [112]:
# bug_df = movie_df_cleaned[movie_df_cleaned['popularity'].map(lambda x: isinstance(x, str))]
# bug_df


In [113]:
# movie_df_cleaned.loc['popularity'] = movie_df_cleaned['popularity'].astype(float)
# movie_df_cleaned

import numpy as np

# movie_df_cleaned['popularity'] = movie_df_cleaned['popularity'].replace(np.nan, 0)
# movie_df_cleaned['popularity'].fillna("nan", inplace=True)

movie_df_cleaned['popularity'] = movie_df_cleaned['popularity'].astype(float)
movie_df_cleaned['revenue'] = movie_df_cleaned['revenue'].astype(float)

movie_df_cleaned['avg_rating'].fillna(0, inplace=True)
movie_df_cleaned['quantity'].fillna(0, inplace=True)

movie_df_cleaned


,budget,id,popularity,revenue,runtime,title,vote_average,vote_count,genre_ids,genres,btc_name,btc_id,genres1,genres2,genres3,genres4,avg_rating,quantity
0,30000000,862,21.946943,373554033.0,81.0,Toy Story,7.7,5415.0,16|35|10751,Animation|Comedy|Family,Toy Story Collection,10194,Animation,Comedy,Family,Animation,3.598930,862
1,65000000,8844,17.015539,262797249.0,104.0,Jumanji,6.9,2413.0,12|14|10751,Adventure|Fantasy|Family,None,None,Adventure,Fantasy,Family,Adventure,3.760163,8844
2,0,15602,11.712900,0.0,101.0,Grumpier Old Men,6.5,92.0,10749|35,Romance|Comedy,Grumpy Old Men Collection,119050,Romance,Comedy,Romance,Romance,0.000000,0
3,16000000,31357,3.859495,81452156.0,127.0,Waiting to Exhale,6.1,34.0,35|18|10749,Comedy|Drama|Romance,None,None,Comedy,Drama,Romance,Comedy,0.000000,0
4,0,11862,8.387519,76578911.0,106.0,Father of the Bride Part II,5.7,173.0,35,Comedy,Father of the Bride Collection,96871,Comedy,Comedy,Comedy,Comedy,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45460,0,439050,0.072051,0.0,90.0,Subdue,4.0,1.0,18|10751,Drama|Family,None,None,Drama,Family,Drama,Drama,0.000000,0
45461,0,111109,0.178241,0.0,360.0,Century of Birthing,9.0,3.0,18,Drama,None,None,Drama,Drama,Drama,Drama,2.583333,111109
45462,0,67758,0.903007,0.0,90.0,Betrayal,3.8,6.0,28|18|53,Action|Drama|Thriller,None,None,Action,Drama,Thriller,Action,0.000000,0
45463,0,227506,0.003503,0.0,87.0,Satan Triumphant,0.0,0.0,,,None,None,,,,,0.000000,0


## 5. Write to CSV


In [114]:
from lib.utils.utils import write_csv

write_csv(DatasetType.movies_metadata.cleaned_path(), movie_df_cleaned)
